In [ ]:
import s3fs
import requests
from urllib import request
from http.cookiejar import CookieJar
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from json import dumps
from io import StringIO
from os.path import dirname, join
import netrc

import os
import sys
import fsspec
import ujson   # fast json
from fsspec_reference_maker.hdf import SingleHdf5ToZarr 
from fsspec_reference_maker.combine import MultiZarrToZarr
import xarray as xr
import dask
import hvplot.xarray
import fsspec_reference_maker
fsspec_reference_maker.__version__

In [ ]:
json_dir = 's3://esip-qhub/nasa/mur/jsons_all/'
json_consolidated_dir = 's3://esip-qhub-public/nasa/mur/'
json_out1 = 'murv41_consolidated_20211005.json'
json_out = './../data/mur_consolidated_tem.json'

In [ ]:
#########Setting up earthdata login credentials 
# this code is from https://github.com/podaac/tutorials/blob/master/notebooks/cloudwebinar/cloud_direct_access_s3.py
def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.
    Valid endpoints:
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print("There's no .netrc file or the The endpoint isn't in the netrc file")

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

###############################################################################
edl="urs.earthdata.nasa.gov"
setup_earthdata_login_auth(edl)

def begin_s3_direct_access():
    url="https://archive.podaac.earthdata.nasa.gov/s3credentials"
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})


In [ ]:
fs2 = fsspec.filesystem('s3', anon=False)  
rpath = json_consolidated_dir + json_out1

In [ ]:
url="https://archive.podaac.earthdata.nasa.gov/s3credentials"
response = requests.get(url).json()

In [ ]:
turl = json_dir+'20021201.20021201090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc.json'

- single file test

In [ ]:
%%time
s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'key':response['accessKeyId'],
          'secret':response['secretAccessKey'],
          'token':response['sessionToken'],
          'client_kwargs':{'region_name':'us-west-2'}
         }

fs = fsspec.filesystem("reference", fo=turl, 
                       ref_storage_args=s_opts,
                       remote_protocol='s3', 
                       remote_options=r_opts,
                       simple_templates=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", consolidated=False)
ds

- Consolidated metadata test (without simple_templates)

In [ ]:
%%time

rpath = json_consolidated_dir + json_out1

s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'key':response['accessKeyId'],
          'secret':response['secretAccessKey'],
          'token':response['sessionToken'],
          'client_kwargs':{'region_name':'us-west-2'}}

fs = fsspec.filesystem("reference", 
                       fo=rpath, 
                       ref_storage_args=s_opts,
                       remote_protocol='s3', 
                       remote_options=r_opts)#,simple_templates=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", consolidated=False)
ds

In [ ]:
%%time

rpath = json_consolidated_dir + json_out1

s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'key':response['accessKeyId'],
          'secret':response['secretAccessKey'],
          'token':response['sessionToken'],
          'client_kwargs':{'region_name':'us-west-2'}}

fs = fsspec.filesystem("reference", 
                       fo=rpath, 
                       ref_storage_args=s_opts,
                       remote_protocol='s3', 
                       remote_options=r_opts
                       ,simple_templates=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", consolidated=False)
ds